In [ ]:
import os
import json
import random
from math import sin, cos, pi, atan2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset
from shapely.geometry import Point, LineString

csv_path = "path_to_livingroom_clusters.csv"
df = pd.read_csv(csv_path)

print("Column names in the csv file:")
print(df.columns.tolist())

label_cols = []
for col in df.columns:
    if "_Type" in col:
        label_cols.append(col)

unique_labels = set()
for col in label_cols:
    for value in df[col].dropna():
        unique_labels.add(value)

label_list = sorted(unique_labels)
label_to_id = {}
for i, label in enumerate(label_list):
    label_to_id[label] = i

print("Total unique labels:", len(label_to_id))
print(label_to_id)

print(os.listdir("path_to_livingroom_heatmaps"))
# ------------------------------------------------------------------

IMAGE_SIZE = 256

def norm_to_px(v_norm: float, img_size: int = IMAGE_SIZE) -> int:
    """
    [-5 … +5] metres  -->  [0 … img_size-1] pixels
    """
    return int((v_norm + 5.0) / 10.0 * img_size)
# ------------------------------------------------------------------

scene_to_objects = {}
for _, row in df.iterrows():
    sid = row["Scene_ID"]
    objs = []
    for i in range(6):
        label     = row[f"Cluster_{i}_Type"]
        x_norm    = row[f"Cluster_{i}_Centroid_X"]
        y_norm    = row[f"Cluster_{i}_Centroid_Y"]
        theta_raw = row[f"Cluster_{i}_Orientations"]

        if pd.isna(label) or pd.isna(x_norm) or pd.isna(y_norm) or pd.isna(theta_raw):
            continue

        x_px = norm_to_px(x_norm)
        y_px = norm_to_px(y_norm)

        theta = None
        if isinstance(theta_raw, str) and ":" in theta_raw:
            for chunk in theta_raw.split(","):
                parts = chunk.strip().split(":")
                if len(parts) >= 3:
                    _, lbl, val = parts
                    if lbl.strip() == label.strip():
                        try:
                            theta = float(val.strip().replace("°", ""))
                            break
                        except ValueError:
                            pass
        if theta is None:
            try:
                theta = float(str(theta_raw).replace("°", ""))
            except ValueError:
                continue

        objs.append({
            "x":     x_px,
            "y":     y_px,
            "theta": theta,
            "label": label
        })

    if objs:
        scene_to_objects[sid] = objs

with open("path_to_wall_data.json") as f:
    wall_data = json.load(f)

print("Parsed scene_to_objects:", len(scene_to_objects))

sid  = random.choice(list(scene_to_objects.keys()))
objs = scene_to_objects[sid]
poly = next(e for e in wall_data if e["scene_id"] == sid)["wall_polyline"]

mask_img = Image.open(next(e for e in wall_data if e["scene_id"] == sid)["room_mask"])
plt.imshow(mask_img, cmap='gray')
for o in objs:
    plt.scatter(o["x"], o["y"], c='r')
plt.title(sid[:8]); plt.gca().invert_yaxis(); plt.show()

def polyline_to_dist(poly, H, W):
    canvas = torch.zeros((H, W))
    pts = torch.tensor(poly).long()
    pts[:, 0] = pts[:, 0].clamp(0, W - 1)
    pts[:, 1] = pts[:, 1].clamp(0, H - 1)
    canvas[pts[:,1], pts[:,0]] = 1.0

    grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing="ij")
    coords = torch.stack([grid_x, grid_y], dim=-1).reshape(-1, 2).float()
    wall_pts = torch.nonzero(canvas).float()

    dist = torch.cdist(coords, wall_pts).min(dim=1)[0].reshape(H, W)
    return dist / dist.max()

heatmap_root = Path("path_to_livingroom_heatmaps")
heatmap_labels = sorted([p.name for p in heatmap_root.iterdir() if p.is_dir()])
print("Heatmap labels:", len(heatmap_labels))


class BedRoomDatasetWithTheta(Dataset):
    def __init__(self, wall_entries, heatmap_folder, heatmap_labels, scene_objects, label_to_id_map):
        self.entries = wall_entries
        self.heatmap_root = Path(heatmap_folder)
        self.labels = heatmap_labels
        self.scene_objects = scene_objects
        self.label_to_id = label_to_id_map

    def __len__(self):
        return len(self.entries)

    def load_mask(self, image_path):
        image = Image.open(image_path).convert("L")
        return torch.from_numpy(np.array(image) > 0).float()

    def load_heatmaps(self, scene_id, height, width):
        heatmap_stack = []
        for label in self.labels:
            heatmap_path = self.heatmap_root / label / f"{scene_id}.png"
            if heatmap_path.exists():
                image = Image.open(heatmap_path).convert("L").resize((width, height))
                heatmap = np.array(image) / 255.0
                heatmap_stack.append(torch.from_numpy(heatmap).float())
            else:
                heatmap_stack.append(torch.zeros((height, width)))
        return torch.stack(heatmap_stack)

    def __getitem__(self, index):
        item = self.entries[index]
        scene_id = item["scene_id"]
        mask_path = item["room_mask"]
        wall_polyline = item["wall_polyline"]

        mask = self.load_mask(mask_path)
        height, width = mask.shape

        wall_distance = polyline_to_dist(wall_polyline, height, width)
        heatmaps = self.load_heatmaps(scene_id, height, width)

        input_tensor = torch.cat([mask.unsqueeze(0), wall_distance.unsqueeze(0), heatmaps], dim=0)

        object_list = self.scene_objects.get(scene_id, [])

        ground_truth = []

        for obj in object_list:
            x = obj["x"]
            y = obj["y"]
            label = obj["label"]

            angle_rad = float(obj["theta"])
            label_index = self.label_to_id.get(label, -1)

            if label_index >= 0:
                ground_truth.append([x, y, angle_rad, label_index])

        target_tensor = torch.tensor(ground_truth, dtype=torch.float32)

        return input_tensor, target_tensor, scene_id


ds = BedRoomDatasetWithTheta(
    wall_entries=wall_data,
    heatmap_folder="path_to_livingroom_heatmaps",
    heatmap_labels=heatmap_labels,
    scene_objects=scene_to_objects,
    label_to_id_map=label_to_id
)

x, objs, sid = ds[0]
print("Scene ID:", sid)
print("Tensor shape:", x.shape)  
print("Ground truth shape:", objs.shape)  
print("Total scenes in the dataset:", len(ds))

In [ ]:
from math import cos, sin, pi, atan2
from shapely.geometry import Point, LineString
import numpy as np

# ────────────────────────────────────────────────────────────
def rule_penality_engine_living(objs, polyline, label_id_to_name):
    wall   = LineString(polyline)
    centre = np.array([128, 128])
    rules  = {f"Rule {i}": 0.0 for i in range(1, 16)}

    def angle_difference(a1, a2):
        return min(abs(a1 - a2), 2 * pi - abs(a1 - a2))

    obj_by_label = {}
    for (x, y, theta, lid) in objs:
        lbl = label_id_to_name.get(int(lid), f"#{lid}")
        obj_by_label.setdefault(lbl, []).append((x, y, theta))

    sofas = sum([obj_by_label.get(k, [])
                 for k in ["sofa", "l_shaped_sofa", "lazy_sofa",
                            "loveseat_sofa", "multi_seat_sofa",
                            "chaise_longue_sofa"]], [])

    coffee_tables = obj_by_label.get("coffee_table", []) + \
                    obj_by_label.get("corner_side_table", []) + \
                    obj_by_label.get("round_end_table", [])

    # Rule 1  Sofa must back a wall (≤60 px) & face room
    for x, y, th in sofas:
        d = Point(x, y).distance(wall)
        if d > 80:
            rules["Rule 1"] += (d - 80) * 1.0
        elif d > 60:
            rules["Rule 1"] += (d - 60) * 0.5
        elif d > 40:
            rules["Rule 1"] += (d - 40) * 0.2

        # face room, not wall
        ahead = Point(x + cos(th) * 15, y + sin(th) * 15)
        if ahead.distance(wall) < d:
            rules["Rule 1"] += 10

    # Rule 2  Coffee-table within 90 px of *some* sofa
    for tx, ty, _ in coffee_tables:
        if sofas:
            d_min = min(np.linalg.norm([tx - sx, ty - sy]) for sx, sy, _ in sofas)
            if d_min > 150:
                rules["Rule 2"] += (d_min - 150) * 0.4
            elif d_min > 120:
                rules["Rule 2"] += (d_min - 120) * 0.2
            elif d_min > 90:
                rules["Rule 2"] += (d_min - 90)  * 0.05

    # Rule 3  Arm / lounge chairs form a conversation area
    chairs = obj_by_label.get("armchair", []) + obj_by_label.get("lounge_chair", [])
    for cx, cy, _ in chairs:
        if sofas:
            d_min = min(np.linalg.norm([cx - sx, cy - sy]) for sx, sy, _ in sofas)
            if d_min > 180:
                rules["Rule 3"] += (d_min - 180) * 0.3
            elif d_min > 140:
                rules["Rule 3"] += (d_min - 140) * 0.15

    # Rule 4  Side / console tables flush to wall
    for lbl in ["console_table", "corner_side_table"]:
        for x, y, th in obj_by_label.get(lbl, []):
            d = Point(x, y).distance(wall)
            if d > 50:
                rules["Rule 4"] += (d - 50) * 0.6
            ahead = Point(x + cos(th) * 10, y + sin(th) * 10)
            if ahead.distance(wall) < d:
                rules["Rule 4"] += 4

    # Rule 5  Bookshelf / cabinet flush to wall
    for lbl in ["bookshelf", "cabinet", "wine_cabinet", "children_cabinet"]:
        for x, y, th in obj_by_label.get(lbl, []):
            d = Point(x, y).distance(wall)
            if d > 60:
                rules["Rule 5"] += (d - 60) * 1.0
            elif d > 40:
                rules["Rule 5"] += (d - 40) * 0.4
            ahead = Point(x + cos(th) * 10, y + sin(th) * 10)
            if ahead.distance(wall) < d:
                rules["Rule 5"] += 5

    # Rule 6  TV-stand placement w.r.t. main sofa
    # - Keep the TV-stand *in front of* the main sofa cluster
    if sofas:
        tvs = obj_by_label.get("tv_stand", [])
        if tvs:
            sx, sy, _   = sofas[0]
            tx, ty, tth = tvs[0]
            dist = np.linalg.norm([sx - tx, sy - ty])
            if dist < 80:
                rules["Rule 6"] += (80 - dist) * 0.5
            elif dist > 180:
                rules["Rule 6"] += (dist - 180) * 0.5

            # orientation check – TV faces sofa-centre
            desired = atan2(sy - ty, sx - tx)
            diff    = angle_difference(desired, tth)

            if diff > 1.0:
                rules["Rule 6"] += 8
            elif diff > 0.7:
                rules["Rule 6"] += 4
            elif diff > 0.4:
                rules["Rule 6"] += 1
        else:
            rules["Rule 6"] += 5


    # Rule 7  Ceiling lamp ≥35 px from wall
    for lx, ly, _ in obj_by_label.get("ceiling_lamp", []):
        d = Point(lx, ly).distance(wall)
        if d < 35:
            rules["Rule 7"] += (35 - d) * 0.8

    # Rule 8  One (and only one) coffee-table preferred
    n_ct = len(coffee_tables)
    if n_ct == 0:
        rules["Rule 8"] += 8
    elif n_ct > 1:
        rules["Rule 8"] += (n_ct - 1) * 5

    # Rule 9  Dining table must have ≥3 chairs within 100 px
    if obj_by_label.get("dining_table"):
        chairs_close = 0
        for dx, dy, _ in obj_by_label["dining_table"]:
            chairs_close += sum(
                np.linalg.norm([dx - cx, dy - cy]) < 100
                for cx, cy, _ in obj_by_label.get("dining_chair", [])
            )
        if chairs_close < 3:
            rules["Rule 9"] += (3 - chairs_close) * 4

    # Rule 10 Collision (<30 px)
    for i in range(len(objs)):
        xi, yi = objs[i][:2]
        for j in range(i + 1, len(objs)):
            xj, yj = objs[j][:2]
            d = np.linalg.norm([xi - xj, yi - yj])
            if d < 10:
                rules["Rule 10"] += (10 - d) * 2
            elif d < 20:
                rules["Rule 10"] += (20 - d) * 1
            elif d < 30:
                rules["Rule 10"] += (30 - d) * 0.3

    # Rule 11  Too many large items in one corner
    corners = 0
    for x, y, _, _ in objs:
        if (x < 60 and y < 60) or (x > 196 and y < 60) or \
           (x < 60 and y > 196) or (x > 196 and y > 196):
            corners += 1
    if corners >= 4:
        rules["Rule 11"] += corners * 3
    elif corners == 3:
        rules["Rule 11"] += corners * 2
    elif corners == 2:
        rules["Rule 11"] += corners * 0.8

    # Rule 12  Isolated object (>120 px from all others)
    if len(objs) > 1:
        for xi, yi, _, _ in objs:
            distances = [np.linalg.norm([xi - xj, yi - yj])
                        for xj, yj, _, _ in objs if (xi, yi) != (xj, yj)]
            if distances:
                nearest = min(distances)
                if nearest > 180:
                    rules["Rule 12"] += (nearest - 180) * 0.4
                elif nearest > 150:
                    rules["Rule 12"] += (nearest - 150) * 0.2
                elif nearest > 120:
                    rules["Rule 12"] += (nearest - 120) * 0.1


    # Rule 13  Wine-cabinet door clearance ≥50 px
    for wx, wy, _ in obj_by_label.get("wine_cabinet", []):
        others = [(ox, oy) for ox, oy, _, lid in objs
                  if label_id_to_name.get(int(lid), "") != "wine_cabinet"]
        if others:
            nearest = min(np.linalg.norm([wx - ox, wy - oy]) for ox, oy in others)
            if nearest < 30:
                rules["Rule 13"] += (30 - nearest) * 1.2
            elif nearest < 40:
                rules["Rule 13"] += (40 - nearest) * 0.6
            elif nearest < 50:
                rules["Rule 13"] += (50 - nearest) * 0.2

    # Rule 14  Ceiling + pendant lamp together
    if obj_by_label.get("ceiling_lamp") and obj_by_label.get("pendant_lamp"):
        rules["Rule 14"] += 7

    # Rule 15  Centre conversation radius: ≥1 seat
    if sofas and chairs:
        sx_mean = np.mean([s[0] for s in sofas])
        sy_mean = np.mean([s[1] for s in sofas])
        near = any(np.linalg.norm([cx - sx_mean, cy - sy_mean]) < 150
                   for cx, cy, _ in chairs)
        if not near:
            rules["Rule 15"] += 6

    return rules



sample_index = 271
tensor_input, object_list, scene_id = ds[sample_index]
scene_entry = next(entry for entry in wall_data if entry["scene_id"] == scene_id)
polyline = scene_entry["wall_polyline"]

label_id_to_name = {v: k for k, v in label_to_id.items()}
print(scene_to_objects["fe174e21-5e11-4004-8347-f4e5e2e7b30c_LivingDiningRoom-20428"])

penalties = rule_penality_engine_living(object_list, polyline, label_id_to_name)

print("Scene ID:", scene_id)
print("Rule-wise Penalties:")
for rule_name, value in penalties.items():
    print(f"{rule_name}: {value:.2f}")
print("\nTotal Penalty:", round(sum(penalties.values()), 2))

room_mask_image = Image.open(scene_entry["room_mask"])
rendered_image = Image.open(scene_entry["rendered_image"])

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(room_mask_image, cmap="gray")
plt.title("Room Mask")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(rendered_image)
plt.title("Rendered Scene")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

csv_path = "path_to_livingroom_clusters.csv"
df = pd.read_csv(csv_path)
cluster_type_cols = [f"Cluster_{i}_Type" for i in range(6)]
df["Num_Valid_Objects"] = df[cluster_type_cols].notna().sum(axis=1)

max_idx   = df["Num_Valid_Objects"].idxmax()
max_scene = df.loc[max_idx]

print(f"Scene with most objects is at index: {max_idx}")
print(f"Scene ID: {max_scene['Scene_ID']}")
print(f"#Objects: {max_scene['Num_Valid_Objects']}")

for i in range(6):
    obj = max_scene[f"Cluster_{i}_Type"]
    if pd.notna(obj):
        print(f"  - Cluster {i}: {obj}")

scene_id = max_scene["Scene_ID"]
print("\nObjects parsed into scene_to_objects:")
# for obj in scene_to_objects.get(scene_id, []):
#     print(obj)

In [ ]:

import random, math, json, os, numpy as np, torch
from torch import nn
from pathlib import Path
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torchvision.models as models


In [ ]:
id_to_label = {v: k for k, v in label_to_id.items()}
print(f"Dataset size: {len(ds)}   |   Number of labels: {len(id_to_label)}")


In [ ]:
def make_empty_sample(idx):
    inp, gt, scene_id = ds[idx]
   
    mask       = inp[0].numpy()
    wall_dist  = inp[1].numpy()
    
    entry      = next(e for e in wall_data if e["scene_id"] == scene_id)
    polyline   = entry["wall_polyline"]
  
    cluster_seq = [int(lid) for *_, lid in gt]
    random.shuffle(cluster_seq)
    
    return mask, wall_dist, polyline, cluster_seq, scene_id


In [ ]:
class LivingRoomPlacementEnv(gym.Env):
    GRID         = 16
    IMG_H = IMG_W = 256
    MAX_STEPS    = 10

    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode=None, sample_idx=None):
        super().__init__()

        self.observation_space = spaces.Box(
            low=0, high=1, shape=(3, self.IMG_H, self.IMG_W), dtype=np.float32
        )
        self.action_space = spaces.Discrete(self.GRID ** 2)

        self.render_mode = render_mode
        self.fixed_idx   = sample_idx
        self.state       = None
        self.steps       = 0

        self._valid_xy   = []       
        self._index_map  = []     

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.steps = 0

        idx = self.fixed_idx if self.fixed_idx is not None else random.randint(0, len(ds)-1)
        mask, wall, poly, cluster_seq, scene_id = make_empty_sample(idx)


        self._valid_xy = []
        idx_to_valid   = [-1]* (self.GRID**2)

        for gi in range(self.GRID**2):
            gx, gy = gi % self.GRID, gi // self.GRID
            x = int((gx+0.5)*self.IMG_W/self.GRID)
            y = int((gy+0.5)*self.IMG_H/self.GRID)
            if mask[y, x] > 0:
                idx_to_valid[gi] = len(self._valid_xy)
                self._valid_xy.append((x, y))

        for gi in range(self.GRID**2):
            if idx_to_valid[gi] != -1:
                continue
            gx, gy = gi % self.GRID, gi // self.GRID
            x = int((gx+0.5)*self.IMG_W/self.GRID)
            y = int((gy+0.5)*self.IMG_H/self.GRID)

            nearest = min(range(len(self._valid_xy)),
                          key=lambda k: (self._valid_xy[k][0]-x)**2 + (self._valid_xy[k][1]-y)**2)
            idx_to_valid[gi] = nearest
        self._index_map = idx_to_valid

        self.state = {
            "mask":      mask,
            "wall":      wall,
            "poly":      poly,
            "scene_id":  scene_id,
            "occupancy": np.zeros_like(mask),
            "placed":    [],
            "remaining": cluster_seq[:self.MAX_STEPS],
        }
        return self._obs(), {}

    def _decode(self, a: int):
        v_idx      = self._index_map[a]
        x, y       = self._valid_xy[v_idx]
        return x, y

    def _obs(self):
        occ = self.state["occupancy"]
        return np.stack([self.state["mask"], self.state["wall"], occ]).astype(np.float32)

    def step(self, action: int):
        self.steps += 1
        if not self.state["remaining"]:
            return self._obs(), 0.0, True, False, {}

        label_id = self.state["remaining"].pop(0)
        x, y     = self._decode(action)
        theta    = 0.0
        self.state["placed"].append((x, y, theta, label_id))
        self.state["occupancy"][y, x] = 1.0

        objs_arr  = np.array(self.state["placed"], dtype=np.float32)
        reward    = -sum(rule_penality_engine_living(objs_arr, self.state["poly"], id_to_label).values())

        done = (not self.state["remaining"]) or (self.steps >= self.MAX_STEPS)
        return self._obs(), reward, done, False, {}

    def render(self):
        if self.render_mode != "human":
            return
        import matplotlib.pyplot as plt
        plt.figure(figsize=(4,4))
        plt.imshow(self.state["mask"], cmap="gray")
        for (x,y,_,lid) in self.state["placed"]:
            plt.scatter([x],[y], s=30)
            plt.text(x, y, id_to_label[lid][:2], color="red", fontsize=6)
        plt.axis("off")
        plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def show_original(env):

    sid = env.state["scene_id"]
    entry = next(e for e in wall_data if e["scene_id"] == sid)
    img   = Image.open(entry["rendered_image"])

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.title(f"Original scene: {sid[:8]}…")
    plt.axis("off")
    plt.show()


In [ ]:
class ResNetFeatureExtractor(BaseFeaturesExtractor):
    """
    :param obs_space: Observation space (Box(C,H,W))
    :param features_dim: Size of the output feature vector
    :param pretrained: Use ImageNet-pretrained weights
    """
    def __init__(self,
                 obs_space: spaces.Box,
                 features_dim: int = 256,
                 pretrained: bool = True):
        super().__init__(obs_space, features_dim)

        resnet = models.resnet18(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        backbone_out = resnet.inplanes
        self.projection = nn.Sequential(
            nn.Flatten(),
            nn.Linear(backbone_out, features_dim),
            nn.ReLU()
        )

        self._features_dim = features_dim

    def forward(self, observations):
        x = self.backbone(observations)
        x = self.avgpool(x)
        return self.projection(x)

In [ ]:
def make_env():
    return LivingRoomPlacementEnv()

vec_env = DummyVecEnv([make_env])
obs = vec_env.reset()
print("Obs shape:", obs.shape, "|  random reward test:", vec_env.step([vec_env.action_space.sample()])[1])


In [ ]:
policy_kwargs = dict(features_extractor_class=ResNetFeatureExtractor, features_extractor_kwargs=dict(features_dim=256, pretrained=True))

model = PPO(
    "CnnPolicy",
    vec_env,
    learning_rate=3e-4,
    n_steps=1024,
    batch_size=64,
    n_epochs=25,
    gamma=0.99,
    policy_kwargs=policy_kwargs,
    verbose=1,
)

TIMESTEPS = 20000     
model.learn(total_timesteps=TIMESTEPS)
model.save("ppo_roomplacer_living_resnet18")
print("MVP training done")


In [ ]:

try:
    label_id_to_name
except NameError:
    label_id_to_name = {v: k for k, v in label_to_id.items()}


In [ ]:
import copy, numpy as np, matplotlib.pyplot as plt, pandas as pd, os

try:
    label_id_to_name
except NameError:
    label_id_to_name = {v: k for k, v in label_to_id.items()}

env_live = LivingRoomPlacementEnv(render_mode="human")
obs_live, _ = env_live.reset()
print("Initial clusters in this room:", len(env_live.state["remaining"]))
print("New room loaded with", len(env_live.state["remaining"]), "objects to go.")

def rollout_candidate(env_base, num_to_place, seed=None):
    rng = np.random.RandomState(seed)
    env_cand = copy.deepcopy(env_base)
    obs = env_cand._obs()
    for _ in range(num_to_place):
        if not env_cand.state["remaining"]:
            break
        act, _ = model.predict(obs, deterministic=False)
        if rng.rand() < 0.1:
            act = env_cand.action_space.sample()
        obs, _, done, _, _ = env_cand.step(int(act))
        if done:
            break
    return env_cand

def total_penalty(env_):
    objs = np.array(env_.state["placed"], dtype=np.float32)
    pen  = rule_penality_engine_living(objs, env_.state["poly"], label_id_to_name)
    return sum(pen.values())

while True:
    try:
        k = int(input("\nHow many NEW objects should I try to place? (0 to quit) ➜ "))
    except ValueError:
        print("Please enter an integer."); continue
    if k <= 0:
        print("Stopping. Final layout shown above."); break
    if not env_live.state["remaining"]:
        print("All objects are already placed!"); break
    k = min(k, len(env_live.state["remaining"]))

    print(f"Generating 5 candidate layouts, each placing {k} objects …")
    candidates = [(total_penalty(env:=rollout_candidate(env_live, k, i)), env) for i in range(5)]
    best_pen, env_live = min(candidates, key=lambda x: x[0])
    obs_live = env_live._obs()

    print(f"Selected candidate with total penalty = {best_pen:.2f}")
    env_live.render()
    show_original(env_live)
    print(f"{len(env_live.state['remaining'])} objects still unplaced.")

    if input("Add more objects? [y/n] ➜ ").lower().strip() != "y":
        print("Interaction finished."); break

print("Saving final_layout_interactive_livingroom.png & placement_log_interactive_livingroom.csv …")
env_live.render()
plt.savefig("final_layout_interactive_livingroom.png", bbox_inches="tight", dpi=200)

pd.DataFrame(env_live.state["placed"],
             columns=["x","y","theta","label_id"]
).to_csv("placement_log_interactive_livingroom.csv", index=False)
print("Files saved in", os.getcwd())
# … your existing end of loop …

print("Saving final_layout_interactive_livingroom.png & placement_log_interactive_livingroom.csv …")
env_live.render()
plt.savefig("final_layout_interactive_livingroom.png", bbox_inches="tight", dpi=200)

placement_log = pd.DataFrame(
    env_live.state["placed"],
    columns=["x","y","theta","label_id"]
)
placement_log.to_csv("placement_log_interactive_livingroom.csv", index=False)
print("Files saved in", os.getcwd())

print("\nFinal placement summary:")
for idx, (x, y, theta, label_id) in enumerate(env_live.state["placed"], start=1):
    label = label_id_to_name[label_id]
    print(f"{idx:2d}. {label:20s} @ (x={x:.1f}, y={y:.1f}), θ={theta:.2f} rad")



In [ ]:
model_path = "ppo_roomplacer_living_resnet18.zip"  
model = PPO.load(model_path)

env = LivingRoomPlacementEnv(render_mode="human")
obs, _ = env.reset()

def compute_penalty(env_instance):
    arr = np.array(env_instance.state["placed"], dtype=np.float32)
    pen = rule_penality_engine_living(arr, env_instance.state["poly"], label_id_to_name)
    return sum(pen.values())

print(f"\nStarting interactive placement; {len(env.state['remaining'])} objects to place.")

while True:
    ans = input("\nPlace one more object? [y/n] ➜ ").strip().lower()
    if ans != "y":
        print("Stopping interactive placement.")
        break

    if not env.state["remaining"]:
        print("No more objects left to place.")
        break
    N_CANDIDATES = 100
    best_pen = float("inf")
    best_env = None

    print(f"Generating {N_CANDIDATES} placement candidates for the next object...")
    for seed in range(N_CANDIDATES):
        cand_env = copy.deepcopy(env)
        obs_cand = cand_env._obs()

        action, _ = model.predict(obs_cand, deterministic=False)
        if np.random.rand() < 0.1:
            action = cand_env.action_space.sample()

        obs_new, _, done, _, _ = cand_env.step(int(action))
        pen = compute_penalty(cand_env)

        if pen < best_pen:
            best_pen = pen
            best_env = cand_env

    env = best_env
    print(f"→ Chose placement with penalty = {best_pen:.2f}")
    env.render()
    print(f"{len(env.state['remaining'])} objects still unplaced.")

print("\nFinal layout:")
env.render()
show_original(env)
plt.savefig("final_layout_interactive_livingroom.png", bbox_inches="tight", dpi=200)

placements = pd.DataFrame(env.state["placed"], columns=["x","y","theta","label_id"])
placements["label"] = placements["label_id"].map(label_id_to_name)
placements.to_csv("placement_log_interactive_livingroom.csv", index=False)

print("\nPlacement log:")
print(placements)

print(f"\nFiles written to {os.getcwd()}:")
print("  - final_layout_interactive_livingroom.png")
print("  - placement_log_interactive_livingroom.csv")

In [ ]:
import pandas as pd

csv_path = "path_to_livingroom_clusters.csv"
df = pd.read_csv(csv_path)

for i in range(6):
    col = f"Cluster_{i}_Orientations"
    if col not in df.columns:
        continue

    print(f"\n=== {col} ===")
    nonnull = df[col].dropna()
    print("Raw sample values:", nonnull.head(10).tolist())

    try:
        nums = nonnull.astype(float)
        print(nums.describe())
    except ValueError:
        print("  (some entries aren’t plain floats; they’ll need string parsing)")
